In [25]:
import os 
import shutil
import cv2
import random

In [2]:
## GATHER ALL TEXTS IN ONE PLACE

DIRPATH = "contents/text-batches"
if not os.path.isdir(DIRPATH):
    os.mkdir(DIRPATH)

counter = 1
for dirpath, dirname, filenames in os.walk(DIRPATH):
    for filename in filenames:
        filepath = f"{dirpath}/{filename}"
        # print(filepath)
        
        if filepath.endswith(".txt"):
            shutil.copy(filepath, f"contents/pre-processing-texts")

            


In [1]:
TXTDIR = 'contents/pre-processing-texts'
txtList = []
for dirpath, dirnames, filenames in os.walk(TXTDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]

In [10]:
## GATHER ALL TEXTS AND PHOTOS TOGETHER

if not os.path.isdir("contents/mixed"):
    os.mkdir("contents/mixed")

TXTDIR = 'contents/pre-processing-texts'
txtList = []
for dirpath, dirnames, filenames in os.walk(TXTDIR):
    for filename in filenames:
        if not filename.endswith(".txt"):
            continue
        fullpath = os.path.join(dirpath,filename)
        shutil.copy(fullpath, "contents/mixed")
# print(txtList)

JPGDIR = 'contents/jpgs'
jpgList = []
for dirpath, dirnames, filenames in os.walk(JPGDIR):
    for filename in filenames:
        if not filename.endswith(".jpg"):
            continue
        fullpath = os.path.join(dirpath,filename)
        shutil.copy(fullpath, "contents/mixed")


# for txt in 

In [ ]:
## MAKE ALL THE PAIRS. WEED OUT THE UNPAIRED TXT FILES
UNPAIREDDIR = "contents/unpaired"
if not os.path.isdir(UNPAIREDDIR):
    os.mkdir(UNPAIREDDIR)

MIXDIR = 'contents/mixed'

txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]
# print(txtList)

jpgList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    jpgList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".jpg")]

# print(jpgList)

for photo in jpgList:
    txtname:str = photo[:-4] + ".txt"
    print(txtname)
    
    if not (txtname in txtList):
        # not paired
        just_file_name = photo.split("/")[-1]
        
        os.rename(photo, f"contents/unpaired/{just_file_name}")
        # shutil.copy(photo, f"contents/unpaired")
        # print(photo)

In [17]:
## CHECK FOR EMPTY LINES

def contains_blank_line(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        content = content.split("\n")
        while not all(content):
            content = content[:-1]
        return content

def strip_blank_line(file_path):
    content = contains_blank_line(file_path)
    with open(file_path,"w") as test:
        for index, line in enumerate(content):
            test.write(line)
            if not index == (len(content)-1):
                test.write("\n")

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]

for file_path in txtList:
    strip_blank_line(file_path=file_path)

In [24]:
## REPLACE STARTING NUMBERS IF NOT ZERO(0)

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]


for file in txtList:
    lines = 0
    line_split: str
    with open(file, "r") as file_inside:
        content = file_inside.read()
        line_split = content.split(' ')

    if not int(line_split[0]) ==0:
        print(line_split)
        with open(file, "w") as f:
            text_to_write = f"0 {line_split[1]} {line_split[2]} {line_split[3]} {line_split[4]}"
            f.write(text_to_write)

In [33]:
## SPLIT ALL PAIRS ACCORDING TO A RATIO

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file[:-4]}' for file in filenames if file.endswith(".txt")]

SPLIT_RATIO = 1 / 4
SPLIT_PROBABILITY = SPLIT_RATIO / (SPLIT_RATIO + 1)
MAX_TEST_SET_SIZE = len(txtList) * SPLIT_PROBABILITY



TESTDIR = "contents/test"
if not os.path.isdir(TESTDIR):
    os.mkdir(TESTDIR)

TRAINDIR = "contents/obj"
if not os.path.isdir(TRAINDIR):
    os.mkdir(TRAINDIR)

test_set_size = 0
for paired_name in txtList:

    if random.random() < SPLIT_PROBABILITY and test_set_size <= MAX_TEST_SET_SIZE:
        shutil.copy(f"{paired_name}.txt","contents/test")
        shutil.copy(f"{paired_name}.jpg","contents/test")
        test_set_size += 1
    else:
        shutil.copy(f"{paired_name}.txt","contents/obj")
        shutil.copy(f"{paired_name}.jpg","contents/obj")

In [ ]:
## GENERATE TEXT FILES FOR THE TRAINING AND TESTING DATA SETS
TESTDIR = "contents/test"
TRAINDIR = "contents/obj"

allTexts = []
for dirpath, dirnames, filenames in os.walk(TESTDIR):
    allTexts += [f"data/test/{textFile}" for textFile in filenames if str(
        textFile[-4:]) == ".jpg"]
print(allTexts)

with open("test.txt", "a") as f:

    for txtFile in allTexts:
        f.write(txtFile)
        f.write("\n")

allTexts = []
for dirpath, dirnames, filenames in os.walk(TRAINDIR):
    allTexts += [f"data/obj/{textFile}" for textFile in filenames if str(
        textFile[-4:]) == ".jpg"]
print(allTexts)

with open("train.txt", "a") as f:

    for txtFile in allTexts:
        f.write(txtFile)
        f.write("\n")

In [40]:
## ARCHIVE OBJ AND TEST, PREPARE FOR UPLOAD
UPLOADDIR = "upload"

if not os.path.isdir(UPLOADDIR):
    os.mkdir(UPLOADDIR)

# shutil.make_archive("upload/obj", "zip", "contents/obj")
shutil.make_archive("upload/test","zip", "contents/test")
os.rename("test.txt", "upload/test.txt")
os.rename("train.txt", "upload/train.txt")
